# Best X finder - Test

Veremos cómo funciona el algoritmo en un script distinto puesto que podría haber variables inicializadas en los test del script *two_best_X_finder.ipynb*

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from termcolor import colored
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent/Best_X_finder


In [4]:
print("Para modificar el tamaño de todos los gráficos")
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,9

Para modificar el tamaño de todos los gráficos


In [5]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Exploración de datos

Estudiaremos de manera general si hay datos faltantes, así como el tipo de datos que contiene

In [7]:
# Años de análisis
starting_year = 2011
period = 11
csv = '.csv'
# Directorios
pitchers_path = 'ETL_data/Free_Agent/Pitchers/free_agents_pitchers_'
hitters_path = 'ETL_data/Free_Agent/Hitters/free_agents_batters_'
# Originales:
df_pitchers = [None]*period
df_pitchers_copy = [None]*period
df_hitters = [None]*period
df_hitters_copy = [None]*period

Leamos todos las bases de datos correspondientes

In [8]:
for year in range(0,period):
    df_pitchers[year] = pd.read_csv(pitchers_path + str(starting_year + year) + csv)
    df_pitchers_copy[year] = df_pitchers[year].copy()
    
    df_hitters[year] = pd.read_csv(hitters_path + str(starting_year + year) + csv)
    df_hitters_copy[year] = df_hitters[year].copy()

Observemos el contenido de las bases de datos

In [9]:
# Pitchers
df_pitchers_copy[5].head()

,Acronimo,Altura,Anio,Anios_contrato,Bateos_pitcher,Bateos_pitcher_2,Cantidad de equipos,Cantidad_agentes_libres,Carreras,Carreras_2,...,Victorias,WAR,WAR_2,WHIP,WHIP_2,WS ganadas,Walks,Walks_2,Wins,Wins_2
0,BOS,6.5,2016,7,227,51529,1,6,106,11236,...,93,2.99,8.9401,1.20,1.4400,8,50,2500,17,289
1,ARI,6.2,2016,6,161,25921,1,7,80,6400,...,68,2.34,5.4756,1.27,1.6129,1,41,1681,13,169
2,DET,6.2,2016,5,118,13924,1,11,63,3969,...,86,0.39,0.1521,1.37,1.8769,4,26,676,9,81
3,SF,0.0,2016,5,190,36100,5,8,88,7744,...,86,2.65,7.0225,1.20,1.4400,8,54,2916,12,144
4,STL,0.0,2016,5,203,41209,2,5,101,10201,...,85,0.37,0.1369,1.32,1.7424,11,30,900,9,81


In [9]:
list(enumerate(df_pitchers_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio'),
 (3, 'Anios_contrato'),
 (4, 'Bateos_pitcher'),
 (5, 'Bateos_pitcher_2'),
 (6, 'Cantidad de equipos'),
 (7, 'Cantidad_agentes_libres'),
 (8, 'Carreras'),
 (9, 'Carreras_2'),
 (10, 'Carreras_ganadas'),
 (11, 'Carreras_ganadas_2'),
 (12, 'Comando'),
 (13, 'Comando_2'),
 (14, 'Control'),
 (15, 'Control_2'),
 (16, 'Dominio'),
 (17, 'Dominio_2'),
 (18, 'ERA'),
 (19, 'ERA_2'),
 (20, 'Edad'),
 (21, 'Equipo'),
 (22, 'Equipo_anterior'),
 (23, 'Estado'),
 (24, 'Inning_pitched'),
 (25, 'Inning_pitched_2'),
 (26, 'Juegos'),
 (27, 'Juegos totales'),
 (28, 'Juegos_2'),
 (29, 'Juegos_iniciados'),
 (30, 'Juegos_iniciados_2'),
 (31, 'Jugador'),
 (32, 'Losses'),
 (33, 'Losses_2'),
 (34, 'Pennants won'),
 (35, 'Peso'),
 (36, 'Playoffs'),
 (37, 'Posicion'),
 (38, 'Promedio_victorias'),
 (39, 'Saves'),
 (40, 'Saves_2'),
 (41, 'Status'),
 (42, 'Strike-outs'),
 (43, 'Strike-outs_2'),
 (44, 'Sueldo'),
 (45, 'Sueldo_base'),
 (46, 'Sueldo_regular'),
 (47, 'TVS'),


In [10]:
# Hitters
df_hitters_copy[5].head()

,Acronimo,Altura,Anio,Anios_contrato,At-bats,At-bats_2,Bateos,Bateos_2,Cantidad de equipos,Cantidad_agentes_libres,...,Sueldo_regular,TVS,TVS_2,Valor_contrato,Valor_contrato_total,Valor_promedio_contrato,Victorias,WAR,WAR_2,WS ganadas
0,BOS,6.5,2016,7,10,100,0,0,1,6,...,30000000,29.72,883.2784,217000000,230000000,31000000,93,2.99,8.9401,8
1,ARI,6.2,2016,6,52,2704,11,121,1,7,...,34000000,16.62,276.2244,206500000,218750000,34416667,68,2.34,5.4756,1
2,CHC,6.5,2016,8,530,280900,122,14884,2,15,...,21666666,4.18,17.4724,184000000,291277500,23000000,102,1.28,1.6384,3
3,BAL,6.3,2016,7,566,320356,125,15625,1,8,...,21065362,41.50,1722.2500,161000000,220260000,23000000,88,3.08,9.4864,3
4,DET,6.1,2016,6,570,324900,140,19600,1,11,...,22125000,25.26,638.0676,132750000,272257500,22125000,86,1.85,3.4225,4


Veamos las columnas con sus índices

In [11]:
list(enumerate(df_hitters_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio'),
 (3, 'Anios_contrato'),
 (4, 'At-bats'),
 (5, 'At-bats_2'),
 (6, 'Bateos'),
 (7, 'Bateos_2'),
 (8, 'Cantidad de equipos'),
 (9, 'Cantidad_agentes_libres'),
 (10, 'Edad'),
 (11, 'Equipo'),
 (12, 'Equipo_anterior'),
 (13, 'Estado'),
 (14, 'Home-runs'),
 (15, 'Home-runs_2'),
 (16, 'Juegos'),
 (17, 'Juegos totales'),
 (18, 'Juegos_2'),
 (19, 'Jugador'),
 (20, 'OPS'),
 (21, 'OPS_2'),
 (22, 'Pennants won'),
 (23, 'Peso'),
 (24, 'Playoffs'),
 (25, 'Porcentaje_bateo'),
 (26, 'Porcentaje_bateo_2'),
 (27, 'Porcetnaje_juegos'),
 (28, 'Porcetnaje_juegos_2'),
 (29, 'Posicion'),
 (30, 'Promedio_victorias'),
 (31, 'RBI'),
 (32, 'RBI_2'),
 (33, 'Status'),
 (34, 'Sueldo'),
 (35, 'Sueldo_base'),
 (36, 'Sueldo_regular'),
 (37, 'TVS'),
 (38, 'TVS_2'),
 (39, 'Valor_contrato'),
 (40, 'Valor_contrato_total'),
 (41, 'Valor_promedio_contrato'),
 (42, 'Victorias'),
 (43, 'WAR'),
 (44, 'WAR_2'),
 (45, 'WS ganadas')]

Veamos si hay bases de datos con variables que distintas entre sí

## Test

In [11]:
def etl_regression(X_raw, Y_raw, df_copy):
    # Creación de la base de datos a transformar las variables de la regresión
    jugador = 'Jugador'
    YX_raw = [None]*period
    
    for year in range(0, period):
        YX_raw[year] = df_copy[year][[jugador, Y_raw, X_raw]]

    # Ajuste de inflación en los salarios    
    change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46,
                                   2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86,
                                   2019:5.99, 2020:4.70, 2021:1}
    for year in range(0, period):
        YX_raw[year][[Y_raw]] = YX_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
        YX_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
        YX_raw[year].rename(columns = {X_raw: X_raw + '_' + str(2011 + year)}, inplace = True)

    # Filtrado de base de datos
    YX_panel_raw = pd.merge(YX_raw[0], YX_raw[1], how = 'outer', on = 'Jugador')

    for year in range(2,period):
        YX_panel_raw = pd.merge(YX_panel_raw, YX_raw[year], how = 'outer', on = 'Jugador')

    YX_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw.sort_index(axis = 1, inplace = True)

    # Recolocación al principio de Jugador
    player_column = YX_panel_raw.pop('Jugador')
    YX_panel_raw.insert(0, 'Jugador', player_column)

    # Creación de base de datos auxiliares  para remover los NaN
    split_column = 1 + period
    max_nan = 2

    if Y_raw > X_raw:
        X_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]

    nulls_quantity = period - max_nan
    X_drop_etl = X_drop_etl.iloc[X_drop_etl[(X_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    X_index_drop = X_drop_etl.index

    if Y_raw > X_raw:
        Y_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    Y_drop_etl = Y_drop_etl.iloc[Y_drop_etl[(Y_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    Y_index_drop = Y_drop_etl.index

    # índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
    index_drop = list(set(X_index_drop).union(set(Y_index_drop)))

    YX_panel_raw.drop(index = index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)

    # Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
    if Y_raw > X_raw:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, 1:split_column].max(axis = 1)
    else:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]].max(axis = 1)
    YX_max_0 = YX_panel_raw[YX_panel_raw[X_raw + '_max'] == 0]
    YX_max_0_index_drop = YX_max_0.index
    YX_panel_raw.drop(columns = X_raw + '_max', inplace = True)
    YX_panel_raw.drop(index = YX_max_0_index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)

    # Variable X
    if Y_raw > X_raw:
        X_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    max_element = X_etl.max(axis = 1)
    min_element = X_etl.min(axis = 1)
    max_min = (max_element + min_element)/2
    dummy_names = []
    X_names = []
    for year in range(0,period):
        dummy_names.append("I_" + str(2011 + year))
        X_names.append("X_" + str(2011 + year))
    for year in range(0,period):
        X_conditions = [
        (X_etl.iloc[:,year] == max_element) & ~(X_etl.iloc[:,year].isnull()),
        (X_etl.iloc[:,year] != max_element) & ~(X_etl.iloc[:,year].isnull())
        ]
        X_conditions_values = [0,1]
        X_etl[dummy_names[year]] = np.select(X_conditions, X_conditions_values, default = np.nan)
    X_aux = X_etl.iloc[:,:period]
    Dummy = X_etl.iloc[:,period:]
    X_max = X_aux.max(axis = 1)
    X_aux[['Max']] = np.sqrt(X_max)
    for year in range(0,period):    
        X_etl['X_' + str(2011 + year)] = \
        (X_aux[X_raw + '_' + str(2011 + year)]*
        ((-1)**Dummy['I_' + str(2011 + year)]))
    X_etl = X_etl.div(X_aux['Max'].values, axis = 0)
    X_etl = X_etl.iloc[:,2*period:]
    for i in range(0,X_etl.shape[0]):
        row = X_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            X_etl.iloc[i,j] = sustitute[j]

    X_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                     'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
    X_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    X_player_column = X_etl.pop('Jugador')
    X_etl.insert(0, 'Jugador', X_player_column)

    # Variable Y
    if Y_raw > X_raw:
        Y_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_etl = YX_panel_raw.iloc[:, 1:split_column]
    for i in range(0,Y_etl.shape[0]):
        row = Y_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl.iloc[i,j] = sustitute[j]

    Y_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                     'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
    Y_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    Y_player_column = Y_etl.pop('Jugador')
    Y_etl.insert(0, 'Jugador', Y_player_column)
    Y_etl_copy = Y_etl.copy()
    y_etl_names = Y_etl_copy['Jugador']
    for i in range(0,len(y_etl_names)):
        row = Y_etl_copy.iloc[i]
        row_length = row.size
        k = 0

        sustitute = []

        for j in range(0, row_length):
            if pd.isna(row[j]) != True:
                k = k + 1

        sustitute.append(y_etl_names[i])
        for j in range(1,k - 1):
            sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl_copy.iloc[i,j] = sustitute[j]

    Y_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                          'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
    Y_etl = Y_etl_copy

    # Base de datos
    regression_list = [None]*(period-1)
    for i in range(0,len(regression_list)):
        merge = pd.merge(Y_etl.iloc[:,[0,i+1]], X_etl.iloc[:,[0,i+1]])
        regression_list[i] = merge
        regression_list[i].columns = ['Jugador', 'Y', 'X']
    df_article_regression = regression_list[0]
    for i in range(1,len(regression_list)):
        df_article_regression = pd.concat([df_article_regression, regression_list[i]])
    df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    df_article_regression.dropna(inplace = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    
    # Construimos la regresion
    Y = df_article_regression['Y'].tolist()
    X = df_article_regression['X'].tolist()
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    
    # Regresamos como argumento al modelo
    return [df_article_regression, model]

In [12]:
X_raw_pitcher = 'Inning_pitched'
Y_raw = 'Sueldo'
df_copy = df_pitchers_copy

In [13]:
df_regression, model = etl_regression(X_raw_pitcher, Y_raw, df_copy)

Veamos la base de datos ocupada para la regresión y después los datos sobre la regresión

In [14]:
df_regression

,Jugador,Y,X
0,A.J. Burnett,-471.040165,14.618481
1,Aaron Harang,509.272111,13.405223
2,Adam Wainwright,313.041401,-3.983648
3,Alex Wood,200.760147,-1.032486
4,Andrew Cashner,-123.85931,12.911235
...,...,...,...
176,Yusmeiro Petit,-202.272138,-9.313981
177,Zach Duke,231.399752,-3.984756
178,Zach Duke,1281.864177,-7.473009
179,Zach Duke,-275.658921,-1.744127


In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7774
Date:                Tue, 20 Sep 2022   Prob (F-statistic):              0.379
Time:                        11:32:35   Log-Likelihood:                -1514.2
No. Observations:                 181   AIC:                             3032.
Df Residuals:                     179   BIC:                             3039.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.7797     78.698     -0.226      0.822    -173.075     137.515
x1            -7.7472      8.787     -0.882      0.379     -25.086       9.592
==============================================================================
Omnibus:                       11.972   Durbin-Watson:                   2.423
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.902
Skew:                          -0.543   Prob(JB):                      0.00158
Kurtosis:                       3.730   Cond. No.                         9.07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Vemos que el algoritmo funciona adecuadamente

## Pitchers

Hagamos una prueba con una serie de estadísticas deportivas para cada grupo

In [16]:
pitcher_statistics = ['Bateos_pitcher', 'Carreras', 'Carreras_ganadas', 'Comando', 'Control',
                      'Dominio', 'ERA', 'Inning_pitched', 'Juegos', 'Juegos_iniciados', 
                      'Losses', 'Saves', 'TVS', 'WAR', 'WHIP', 'Walks', 'Wins', 
                      'Bateos_pitcher_2', 'Carreras_2', 'Carreras_ganadas_2', 'Comando_2', 'Control_2',
                      'Dominio_2', 'ERA_2', 'Inning_pitched_2', 'Juegos_2', 'Juegos_iniciados_2', 
                      'Losses_2', 'Saves_2', 'TVS_2', 'WAR_2', 'WHIP_2', 'Walks_2', 'Wins_2']
Y_pitcher_raw = 'Sueldo'
pitcher_Statistics_len = len(pitcher_statistics)
pitcher_model = [None]*pitcher_Statistics_len
df_pitcher_model = [None]*pitcher_Statistics_len

In [18]:
for statistic in range(0,pitcher_Statistics_len):
    df_pitcher_model[statistic], pitcher_model[statistic] = etl_regression(pitcher_statistics[statistic], Y_raw, df_pitchers_copy)
    
    print(colored(pitcher_statistics[statistic], "cyan"))
    print(pitcher_model[statistic].summary())
    print('\n')

Bateos_pitcher
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     7.683
Date:                Mon, 19 Sep 2022   Prob (F-statistic):            0.00616
Time:                        11:44:46   Log-Likelihood:                -1510.7
No. Observations:                 181   AIC:                             3025.
Df Residuals:                     179   BIC:                             3032.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.4434     76.756     

## Hitters

Hagamos lo mismo para los *hitters*.

In [17]:
hitter_statistics = ['At-bats', 'Bateos', 'Home-runs', 'Juegos',
                     'OPS', 'Porcentaje_bateo', 'Porcetnaje_juegos',
                     'RBI', 'TVS', 'WAR', 'At-bats_2', 'Bateos_2',
                     'Home-runs_2', 'Juegos_2', 'OPS_2',
                     'Porcentaje_bateo_2', 'Porcetnaje_juegos_2',
                     'RBI_2', 'TVS_2', 'WAR_2']
Y_hitter_raw = 'Sueldo'
hitter_Statistics_len = len(hitter_statistics)
hitter_model = [None]*hitter_Statistics_len
df_hitter_model = [None]*hitter_Statistics_len

In [18]:
for statistic in range(0,hitter_Statistics_len):
    df_hitter_model[statistic], hitter_model[statistic] = etl_regression(hitter_statistics[statistic], Y_raw, df_hitters_copy)
    
    print(colored(hitter_statistics[statistic], "cyan"))
    print(hitter_model[statistic].summary())
    print('\n')

At-bats
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     17.03
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           5.43e-05
Time:                        11:32:42   Log-Likelihood:                -1677.7
No. Observations:                 200   AIC:                             3359.
Df Residuals:                     198   BIC:                             3366.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -142.3605     77.533     -1.836 